<a href="https://colab.research.google.com/github/dumpling2/steamhistory/blob/main/Steam%E3%82%B2%E3%83%BC%E3%83%A0%E3%83%AA%E3%82%B9%E3%83%88%E5%8F%96%E5%BE%97%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# Steam APIを使用して所有ゲームリスト（最終プレイ日時、ジャンル、カテゴリ付き）を取得するPythonスクリプト
# --- Steam APIキーの取得方法 ---
# 1. Steam Developersポータルにアクセス: https://steamcommunity.com/dev/apikey
# 2. Steamアカウントでログイン
# 3. "Register for a Steam Web API Key" をクリック
# 4. ドメイン名を入力 (任意)
# 5. 利用規約に同意して "Register" をクリック
# 6. 表示された "Key" がSteam APIキーです
# ---

# --- SteamIDの取得方法 ---
# SteamIDには、SteamID64、SteamID3、SteamID32など、複数の種類があります。
# このスクリプトでは、SteamID64を使用します。
# SteamID64は、SteamプロフィールページのURLから確認できます。
# 例：https://steamcommunity.com/profiles/76561198000000001/
# 上記のURLの場合、SteamID64は "76561198000000001" です。
# ---

import requests
import json
import sys # エラー出力用にインポート
import datetime # Unixタイムスタンプ変換用にインポート
import time # time.sleepとローカルタイムゾーン取得用にインポート
from contextlib import contextmanager
import os # 環境変数読み込み用 (Colab外実行時)

# 標準出力を一時的にファイルにリダイレクトするコンテキストマネージャ
@contextmanager
def redirect_stdout(filepath):
    """標準出力を一時的にファイルにリダイレクトするコンテキストマネージャ"""
    original_stdout = sys.stdout
    try:
        # ファイルを新規書き込み ('w') で開く
        with open(filepath, 'w', encoding='utf-8') as f:
            sys.stdout = f
            yield
    finally:
        sys.stdout = original_stdout


# --- APIリクエスト (GetOwnedGames) ---
def get_owned_games(api_key, steam_id, include_appinfo, include_played_free_games):
    """Steam APIにリクエストを送信し、所有ゲームの基本データを取得する関数"""
    api_url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
    params = {
        "key": api_key,
        "steamid": steam_id,
        "format": "json",
        "include_appinfo": "1" if include_appinfo else "0",
        "include_played_free_games": "1" if include_played_free_games else "0",
    }

    # このプリントはコンソール（Colabのセル出力）に表示されます
    print("GetOwnedGames APIに接続中...")
    try:
        response = requests.get(api_url, params=params, timeout=30) # タイムアウトを30秒に設定
        response.raise_for_status() # HTTPエラーがあれば例外を発生させる (4xx, 5xx)
        print("GetOwnedGames APIからのデータ取得に成功しました。")
        return response.json() # JSONレスポンスを返す
    except requests.exceptions.Timeout:
        print("エラー: GetOwnedGames APIへの接続がタイムアウトしました。", file=sys.stderr)
        return None
    except requests.exceptions.HTTPError as e:
        print(f"エラー: HTTPエラーが発生しました ({e.response.status_code})。", file=sys.stderr)
        print(f"レスポンス内容: {e.response.text}", file=sys.stderr)
        if e.response.status_code in [401, 403]:
            print("APIキーまたはSteamIDが無効か、アクセス権限がない可能性があります。", file=sys.stderr)
        return None
    except requests.exceptions.RequestException as e:
        print(f"エラー: GetOwnedGames APIリクエスト中に問題が発生しました: {e}", file=sys.stderr)
        return None
    except json.JSONDecodeError:
        print("エラー: GetOwnedGames APIレスポンスのJSON解析に失敗しました。", file=sys.stderr)
        try:
            print(f"受信したテキスト: {response.text}", file=sys.stderr)
        except NameError:
            pass
        return None

# --- Steam App Details APIリクエスト (個別取得用) ---
def get_single_app_details(appid):
    """指定されたAppIDのゲーム詳細データを取得する関数"""
    # APIキーはここでは不要 (公開情報のため)
    api_url = "https://store.steampowered.com/api/appdetails/"
    params = {
        "appids": appid,
        "cc": "jp", # 日本の価格や言語情報を優先
        "l": "japanese" # 可能であれば日本語の説明を取得
    }
    # 注意: filters パラメータは不安定なため使用しません

    # このprintはファイルに出力されます (redirect_stdout内から呼び出されるため)
    # print(f"   AppID {appid}: App Details APIに接続中...") # 詳細すぎるのでコメントアウト
    try:
        response = requests.get(api_url, params=params, timeout=15) # 個別取得なのでタイムアウトを少し短く
        response.raise_for_status()
        data = response.json()
        # APIが成功したかどうかのチェック
        # dataがNoneでないこと、appidがキーとして存在すること、successがTrueであることを確認
        if data and str(appid) in data and data[str(appid)].get("success"):
            return data[str(appid)].get("data") # "data" 部分のみ返す
        else:
            # 失敗またはデータなしの場合のログ（ファイルに出力される）
            # print(f"   AppID {appid}: APIレスポンスで success=false または AppIDが見つかりません。")
            # print(f"   レスポンス: {data}") # デバッグ用にレスポンス内容を表示
            return None
    except requests.exceptions.Timeout:
        # sys.stderr に出力することでコンソールに表示される
        print(f"警告: AppID {appid}: App Details API タイムアウト", file=sys.stderr)
        return None
    except requests.exceptions.HTTPError as e:
        # 404 Not Foundなどはゲームが存在しない場合などでありうるので、警告レベルでコンソールに表示
        if e.response.status_code == 404:
             print(f"情報: AppID {appid}: App Details が見つかりません (404)。非公開、削除、または地域制限の可能性があります。", file=sys.stderr)
        elif e.response.status_code == 429:
             print(f"警告: AppID {appid}: App Details API レート制限 (429)。待機時間を長くすることを検討してください。", file=sys.stderr)
        else:
            print(f"警告: AppID {appid}: App Details API HTTPエラー ({e.response.status_code})。", file=sys.stderr)
        return None
    except requests.exceptions.RequestException as e:
        print(f"警告: AppID {appid}: App Details API リクエストエラー: {e}", file=sys.stderr)
        return None
    except json.JSONDecodeError:
        print(f"警告: AppID {appid}: App Details API JSONデコードエラー", file=sys.stderr)
        try:
            # response変数が定義されている場合のみテキストを表示
            if 'response' in locals():
                print(f"   受信したテキスト: {response.text}", file=sys.stderr)
        except Exception: # response.textへのアクセスエラーも考慮
            pass
        return None
    except Exception as e:
        # 予期せぬエラーをキャッチ
        print(f"警告: AppID {appid}: App Details 取得中に予期せぬエラー: {e}", file=sys.stderr)
        return None


# --- ヘルパー関数 ---
def format_timestamp(timestamp):
    """Unixタイムスタンプをローカルの日時文字列に変換する"""
    if timestamp is None or timestamp == 0:
        return "プレイ記録なし"
    try:
        # ローカルタイムゾーンを考慮して変換
        dt_object = datetime.datetime.fromtimestamp(timestamp)
        return dt_object.strftime('%Y-%m-%d %H:%M:%S') # 例: 2023-10-27 15:30:00
    except ValueError:
        # タイムスタンプが不正な場合 (例: 負の値)
        return "無効な日時"
    except Exception as e:
        print(f"日時変換エラー: {e}", file=sys.stderr) # エラー詳細をコンソールに表示
        return "日時変換エラー"

# --- メイン処理 ---
if __name__ == "__main__":
    # Google Colab環境かどうかを判定
    is_colab = 'google.colab' in sys.modules

    if is_colab:
        try:
            from google.colab import userdata
            # APIキーとSteamID64を読み込む
            API_KEY = userdata.get('API_KEY')
            STEAM_ID = userdata.get('STEAM_ID')
        except ImportError:
            print("エラー: Google Colab環境のはずですが、userdataモジュールをインポートできません。", file=sys.stderr)
            sys.exit(1)
        except Exception as e:
             print(f"エラー: Google Colabのuserdata読み込み中にエラーが発生しました: {e}", file=sys.stderr)
             sys.exit(1)
    else:
        print("Google Colab環境外で実行します。環境変数からAPIキーとSteamIDを取得します。", file=sys.stderr)
        # ローカル環境などで実行する場合、環境変数などから読み込む
        API_KEY = os.environ.get('STEAM_API_KEY') # 環境変数名を適宜変更してください
        STEAM_ID = os.environ.get('STEAM_ID') # 環境変数名を適宜変更してください

    # APIキーとSteamID64が設定されていない場合のエラー処理
    if not API_KEY or not STEAM_ID:
        print("エラー: APIキーまたはSteamIDが設定されていません。", file=sys.stderr)
        if is_colab:
            print("以下のコードを実行して、API_KEYとSTEAM_IDをユーザーシークレットに設定してください:", file=sys.stderr)
            # コードブロックが正しく表示されるようにインデントを調整
            print("""
from google.colab import userdata
userdata.set('API_KEY', 'YOUR_API_KEY')
userdata.set('STEAM_ID', 'YOUR_STEAM_ID64')
""", file=sys.stderr)
        else:
             print("環境変数 STEAM_API_KEY と STEAM_ID を設定してください。", file=sys.stderr)
        sys.exit(1)

    # --- 設定 ---
    INCLUDE_APPINFO = True # GetOwnedGamesで基本情報を取得するかどうか (名前取得に必要)
    INCLUDE_PLAYED_FREE_GAMES = True # 無料ゲームを含めるか
    SHOW_ZERO_PLAYTIME_GAMES = False # プレイ時間0分を表示するか
    SORT_BY_PLAYTIME = False # プレイ時間でソートするか
    SORT_BY_NAME = False # 名前でソートするか (SORT_BY_PLAYTIME=Falseの場合のみ有効)
    SORT_BY_LAST_PLAYED = True # 最終プレイ日時でソートするか (最優先)
    # APIレート制限対策の待機時間 (秒) - 必要に応じて調整 (1.5秒以上推奨)
    SLEEP_DURATION = 1.6
    # 出力ファイル名
    output_filename = 'output_detailed.txt'

    # GetOwnedGames APIから基本データを取得
    game_data = get_owned_games(API_KEY, STEAM_ID, INCLUDE_APPINFO, INCLUDE_PLAYED_FREE_GAMES)

    # 標準出力をファイルにリダイレクト
    with redirect_stdout(output_filename):
        if game_data and "response" in game_data and "games" in game_data["response"]:
            games_list = game_data["response"]["games"]
            game_count_total = game_data["response"].get("game_count", len(games_list))

            print(f"\n--- 所有ゲームリスト (合計: {game_count_total} 件) ---")

            # プレイ時間0分のゲームを除外
            if not SHOW_ZERO_PLAYTIME_GAMES:
                original_count = len(games_list)
                games_list = [game for game in games_list if game.get("playtime_forever", 0) > 0]
                filtered_count = original_count - len(games_list)
                if filtered_count > 0:
                    print(f"(プレイ時間0分のゲーム {filtered_count} 件を非表示にしました)")

            # --- ソート処理 ---
            sort_description = ""
            if SORT_BY_LAST_PLAYED:
                games_list.sort(key=lambda x: x.get("rtime_last_played", 0), reverse=True)
                sort_description = "(最終プレイ日時順にソートしました)"
            elif SORT_BY_PLAYTIME:
                games_list.sort(key=lambda x: x.get("playtime_forever", 0), reverse=True)
                sort_description = "(プレイ時間順にソートしました)"
            elif SORT_BY_NAME and INCLUDE_APPINFO:
                # 名前のないゲーム (AppID表示) が最後に来るように調整
                games_list.sort(key=lambda x: x.get("name", "\uffff")) # \uffff は非常に大きいUnicode文字
                sort_description = "(ゲーム名順にソートしました)"

            if sort_description:
                print(sort_description)

            game_count_processed = len(games_list) # フィルターおよびソート後のゲーム数
            print(f"詳細情報を取得するゲーム数: {game_count_processed} 件")
            # 処理開始のメッセージはファイルにも書く
            print("App Details APIへの接続を開始します (レート制限のため時間がかかります)...")
            # コンソールにも処理開始を知らせる (ファイルリダイレクト前に表示)
            print("\n詳細情報の取得を開始します。進捗はファイルに出力されます...", file=sys.stderr)


            if not games_list:
                print("表示対象のゲームが見つかりませんでした。")
            else:
                # --- メインループ ---
                for i, game in enumerate(games_list):
                    appid = game.get("appid") # appidがないケースは考えにくいが一応getを使う
                    # appidがない場合はスキップ
                    if not appid:
                        print(f"\n警告: {i+1}/{game_count_processed} - AppIDが見つからないためスキップします。ゲームデータ: {game}", file=sys.stderr)
                        continue

                    name = game.get("name", f"不明 (AppID: {appid})")
                    playtime_forever = game.get("playtime_forever", 0)
                    playtime_hours = playtime_forever / 60
                    last_played_timestamp = game.get("rtime_last_played")
                    last_played_formatted = format_timestamp(last_played_timestamp)

                    # ジャンルとカテゴリを初期化
                    genre_str = "情報なし"
                    category_str = "情報なし" # 「タグ」の代わりに「カテゴリ」とする

                    # --- ループ内で個別にApp Detailsを取得 ---
                    # print(f"   AppID {appid}: 詳細取得中...") # 詳細すぎるのでコメントアウト
                    app_details = get_single_app_details(appid)

                    if app_details:
                        # ジャンルの取得
                        genres_list = app_details.get('genres')
                        if genres_list and isinstance(genres_list, list): # リストであることを確認
                            # descriptionが存在し、Noneでない文字列のみを抽出
                            descriptions = [g.get("description") for g in genres_list if isinstance(g, dict) and g.get("description")]
                            if descriptions:
                                genre_str = ", ".join(descriptions)
                        elif genres_list:
                             print(f"   AppID {appid}: ジャンルデータが予期しない形式です: {genres_list}", file=sys.stderr)


                        # カテゴリの取得 (タグの代わり)
                        categories_list = app_details.get('categories')
                        if categories_list and isinstance(categories_list, list): # リストであることを確認
                            # descriptionが存在し、Noneでない文字列のみを抽出
                            descriptions = [c.get("description") for c in categories_list if isinstance(c, dict) and c.get("description")]
                            if descriptions:
                                # 必要に応じて表示するカテゴリをフィルタリングしても良い
                                # 例: ["シングルプレイヤー", "マルチプレイヤー", "実績"] のみ表示するなど
                                category_str = ", ".join(descriptions)
                        elif categories_list:
                             print(f"   AppID {appid}: カテゴリデータが予期しない形式です: {categories_list}", file=sys.stderr)

                    # else:
                        # app_detailsがNoneの場合 (APIエラーやデータなし)
                        # print(f"   AppID {appid}: 詳細情報の取得に失敗またはデータなし。") # get_single_app_details内でログ出力済

                    # --- 結果の表示 (ファイルへ) ---
                    print(f"\n{i+1}/{game_count_processed}. {name}") # 進捗表示を追加
                    print(f"   AppID: {appid}")
                    if "playtime_forever" in game:
                        print(f"   総プレイ時間: {playtime_hours:.2f} 時間 ({playtime_forever} 分)")
                    else:
                        print("   総プレイ時間: 情報なし")
                    print(f"   最終プレイ: {last_played_formatted}")
                    print(f"   ジャンル: {genre_str}")
                    print(f"   カテゴリ: {category_str}") # "タグ" を "カテゴリ" に変更

                    # --- レート制限対策の待機 ---
                    # print(f"   待機中 ({SLEEP_DURATION}秒)...") # 詳細すぎるのでコメントアウト
                    time.sleep(SLEEP_DURATION)

        elif game_data and "response" in game_data and not game_data["response"]:
            # ゲームリストが空の場合 (プライバシー設定など)
            print("\nゲームリストを取得できませんでした。")
            print("Steamプロフィールのゲーム詳細が非公開になっている可能性があります。")
            print("または、指定したSteamIDにゲームが登録されていない可能性もあります。")
            print("Steamプロフィール設定を確認してください: https://steamcommunity.com/my/edit/settings")
        elif game_data is None:
            # get_owned_games でエラーが発生した場合
            print("\nGetOwnedGames APIからのデータ取得に失敗したため、処理を中断しました。")
            print("エラーメッセージを確認してください。")
        else:
            # 予期しないレスポンス形式
            print("\nエラー: GetOwnedGames APIから予期しない形式のレスポンスを受け取りました。", file=sys.stderr)
            print(f"受信データ: {game_data}", file=sys.stderr)

        print("\n--- 処理終了 ---")

    # コンソールへの完了メッセージ (ファイルリダイレクトの外)
    print(f"\n処理が完了し、結果が {output_filename} に保存されました。")
    # エラーがあったかもしれないことを示唆
    print("処理中に警告やエラーが表示された場合は、コンソール出力も確認してください。")

GetOwnedGames APIに接続中...
GetOwnedGames APIからのデータ取得に成功しました。



詳細情報の取得を開始します。進捗はファイルに出力されます...



処理が完了し、結果が output_detailed.txt に保存されました。
処理中に警告やエラーが表示された場合は、コンソール出力も確認してください。


In [1]:
# --- Steam APIキーとSteamIDの取得方法 ---
# 1. Steam APIキー:
#    - Steam Developersポータル (https://steamcommunity.com/dev/apikey) にアクセス
#    - Steamアカウントでログイン
#    - "Register for a Steam Web API Key" をクリック
#    - ドメイン名を入力 (任意)
#    - 利用規約に同意して "Register" をクリック
#    - 表示された "Key" がSteam APIキーです
# 2. SteamID64:
#    - SteamプロフィールページのURLから確認できます。
#    - 例：https://steamcommunity.com/profiles/76561198000000001/
#    - 上記のURLの場合、SteamID64は "76561198000000001" です。
# ---

import requests
import json
import sys
import datetime
import time
import os
from contextlib import contextmanager
from google.colab import userdata

# APIリクエストのエラー処理を共通化
def handle_api_error(response, api_name, app_id=None):
    """APIリクエストのエラー処理を共通化"""
    try:
        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        message = f"エラー: {api_name} APIへの接続がタイムアウトしました。"
        if app_id:
            message = f"警告: AppID {app_id}: {api_name} API タイムアウト"
        print(message, file=sys.stderr)
        return None
    except requests.exceptions.HTTPError as e:
        message = f"エラー: {api_name} APIでHTTPエラーが発生しました ({e.response.status_code})."
        if app_id:
            if e.response.status_code == 404:
                message = f"情報: AppID {app_id}: {api_name} が見つかりません (404)."
            elif e.response.status_code == 429:
                message = f"警告: AppID {app_id}: {api_name} API レート制限 (429)."
            else:
                message = f"警告: AppID {app_id}: {api_name} API HTTPエラー ({e.response.status_code})."
        print(message, file=sys.stderr)
        return None
    except requests.exceptions.RequestException as e:
        message = f"エラー: {api_name} APIリクエスト中に問題が発生しました: {e}"
        if app_id:
            message = f"警告: AppID {app_id}: {api_name} API リクエストエラー: {e}"
        print(message, file=sys.stderr)
        return None
    except json.JSONDecodeError:
        message = f"エラー: {api_name} APIレスポンスのJSON解析に失敗しました。"
        if app_id:
            message = f"警告: AppID {app_id}: {api_name} API JSONデコードエラー"
        print(message, file=sys.stderr)
        return None

# --- APIリクエスト関数 ---
def get_owned_games(api_key, steam_id, include_appinfo=True, include_played_free_games=True):
    """Steam APIにリクエストを送信し、所有ゲームの基本データを取得する関数"""
    api_url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
    params = {
        "key": api_key,
        "steamid": steam_id,
        "format": "json",
        "include_appinfo": "1" if include_appinfo else "0",
        "include_played_free_games": "1" if include_played_free_games else "0",
    }
    print("GetOwnedGames APIに接続中...")
    response = requests.get(api_url, params=params, timeout=30)
    return handle_api_error(response, "GetOwnedGames")

def get_app_details(app_id):
    """指定されたAppIDのゲーム詳細データを取得する関数"""
    api_url = "https://store.steampowered.com/api/appdetails/"
    params = {
        "appids": app_id,
        "cc": "jp",
        "l": "japanese"
    }
    response = requests.get(api_url, params=params, timeout=15)
    data = handle_api_error(response, "App Details", app_id)
    if data and str(app_id) in data and data[str(app_id)].get("success"):
        return data[str(app_id)].get("data")
    return None

# --- ヘルパー関数 ---
def format_timestamp(timestamp):
    """Unixタイムスタンプをローカルの日時文字列に変換する"""
    return "プレイ記録なし" if timestamp is None or timestamp == 0 else datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

@contextmanager
def redirect_stdout(filepath):
    """標準出力を一時的にファイルにリダイレクトするコンテキストマネージャ"""
    original_stdout = sys.stdout
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            sys.stdout = f
            yield
    finally:
        sys.stdout = original_stdout

# --- メイン処理 ---
def main():
    is_colab = 'google.colab' in sys.modules
    api_key = userdata.get('API_KEY') if is_colab else os.environ.get('STEAM_API_KEY')
    steam_id = userdata.get('STEAM_ID') if is_colab else os.environ.get('STEAM_ID')

    if not api_key or not steam_id:
        print("エラー: APIキーまたはSteamIDが設定されていません。", file=sys.stderr)
        print("Google Colabで実行する場合は、以下のコードを実行してuserdataに設定してください:", file=sys.stderr)
        print("""
from google.colab import userdata
userdata.set('API_KEY', 'YOUR_API_KEY')
userdata.set('STEAM_ID', 'YOUR_STEAM_ID64')
""", file=sys.stderr)
        return

    output_filename = 'output_detailed.txt'
    sleep_duration = 1.6

    game_data = get_owned_games(api_key, steam_id)

    with redirect_stdout(output_filename):
        if game_data and "response" in game_data and "games" in game_data["response"]:
            games = game_data["response"]["games"]
            total_games = game_data["response"].get("game_count", len(games))
            print(f"\n--- 所有ゲームリスト (合計: {total_games} 件) ---")

            games = [game for game in games if game.get("playtime_forever", 0) > 0]
            filtered_count = total_games - len(games)
            if filtered_count > 0:
                print(f"(プレイ時間0分のゲーム {filtered_count} 件を非表示にしました)")

            # ソート
            games.sort(key=lambda x: x.get("rtime_last_played", 0), reverse=True)
            print("(最終プレイ日時順にソートしました)")

            print(f"詳細情報を取得するゲーム数: {len(games)} 件")
            print("App Details APIへの接続を開始します (レート制限のため時間がかかります)...")
            print("\n詳細情報の取得を開始します。進捗はファイルに出力されます...", file=sys.stderr)

            for i, game in enumerate(games):
                app_id = game.get("appid")
                if not app_id:
                    print(f"\n警告: {i+1}/{len(games)} - AppIDが見つからないためスキップします。ゲームデータ: {game}", file=sys.stderr)
                    continue

                name = game.get("name", f"不明 (AppID: {app_id})")
                playtime_hours = game.get("playtime_forever", 0) / 60
                last_played_formatted = format_timestamp(game.get("rtime_last_played"))

                app_details = get_app_details(app_id)

                genre_str = "情報なし"
                category_str = "情報なし"

                if app_details:
                    genres = app_details.get('genres', [])
                    genre_str = ", ".join([g.get("description") for g in genres if isinstance(g, dict) and g.get("description")])

                    categories = app_details.get('categories', [])
                    category_str = ", ".join([c.get("description") for c in categories if isinstance(c, dict) and c.get("description")])

                print(f"\n{i+1}/{len(games)}. {name}")
                print(f"   AppID: {app_id}")
                print(f"   総プレイ時間: {playtime_hours:.2f} 時間 ({game.get('playtime_forever', 0)} 分)")
                print(f"   最終プレイ: {last_played_formatted}")
                print(f"   ジャンル: {genre_str}")
                print(f"   カテゴリ: {category_str}")
                time.sleep(sleep_duration)

        elif game_data and "response" in game_data and not game_data["response"]:
            print("\nゲームリストを取得できませんでした。Steamプロフィールのゲーム詳細が非公開になっている可能性があります。", file=sys.stderr)
        elif game_data is None:
            print("\nGetOwnedGames APIからのデータ取得に失敗したため、処理を中断しました。エラーメッセージを確認してください。", file=sys.stderr)
        else:
            print("\nエラー: GetOwnedGames APIから予期しない形式のレスポンスを受け取りました。", file=sys.stderr)
            print(f"受信データ: {game_data}", file=sys.stderr)

        print("\n--- 処理終了 ---")

    print(f"\n処理が完了し、結果が {output_filename} に保存されました。", file=sys.stderr)
    print("処理中に警告やエラーが表示された場合は、コンソール出力も確認してください。", file=sys.stderr)

if __name__ == "__main__":
    main()

GetOwnedGames APIに接続中...



詳細情報の取得を開始します。進捗はファイルに出力されます...

処理が完了し、結果が output_detailed.txt に保存されました。
処理中に警告やエラーが表示された場合は、コンソール出力も確認してください。
